# Encoding Known Face Database
The following codes are based on the following LinkedIn Learning course: https://www.linkedin.com/learning/deep-learning-face-recognition

The face dataset is obtained from Labelled Faces in the Wild (LFW):
https://www.kaggle.com/jessicali9530/lfw-dataset


---
### Mount to Google Drive
Mount to your Google Drive to access the images and model files.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

dataset_location = "/content/drive/My Drive/Crafting/AdvCVDL/dataset/"

---
## Import the required libraries

* Remove the comment if the import fails.
* put back the comment after installing the library

In [ ]:
!pip install dlib
!pip install face_recognition

import face_recognition
import os
import matplotlib.pyplot as plt
import pandas as pd
from imutils import paths

     |████████████████████████████████| 100.2MB 42kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=75c4cebdc61b2fa3f6f5237fadcb2ca1aa8dcc26ae155bdd0dad87099dad4710
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


---
## Load the csv file with the Face Encodings

In [ ]:
face_encodings_csv = dataset_location+"Face/KnownFaceEncodings.csv"
# Load the csv file with the Face Encodings
Known_db = pd.read_csv(face_encodings_csv)

print(Known_db.shape)

---
## Encode known faces from a folder

In [ ]:
# grab the image paths
print("[INFO] loading face images...")
imagePaths = sorted(list(paths.list_images(dataset_location+"Face/LFW_Train")))

# loop over the input images
for ind, imagePath in enumerate(imagePaths):
	# load the image, pre-process it, and store it in the data list
  name = imagePath.split(os.path.sep)[-1]
  name = name.split(".")[0]
  image = face_recognition.load_image_file(imagePath)

  # Generate the face encodings
  face_encodings = face_recognition.face_encodings(image)

  if len(face_encodings) == 0:
      # No faces found in the image.
      print("No faces were found.")
  else:
      # Grab the first face encoding
      first_face_encoding = face_encodings[0]
      data = first_face_encoding.tolist()
      data.insert(0,name)
      data_df = pd.DataFrame([data], columns=Known_db.columns)
      Known_db = Known_db.append(data_df)
      print(ind,name,"- Added")


---
## Encode a face from webcam

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
name = input("Enter your name: ")

try:
  filename = take_photo(dataset_location+"temp.jpg")
except Exception as err:
  print(str(err))


image = face_recognition.load_image_file(filename)

# Generate the face encodings
face_encodings = face_recognition.face_encodings(image)

if len(face_encodings) == 0:
    # No faces found in the image.
    print("No faces were found.")
else:
    # Grab the first face encoding
    first_face_encoding = face_encodings[0]
    data = first_face_encoding.tolist()
    data.insert(0,name)
    data_df = pd.DataFrame([data], columns=Known_db.columns)
    Known_db = Known_db.append(data_df)
    print(name,"- Added")

---
## Safe the Face Encodings to a csv file

In [ ]:
# Save the Face Encodings to a csv file
Known_db.to_csv(face_encodings_csv, index=False)